In [ ]:
import sys
import os
import numpy as np
import cv2
from matplotlib import pyplot as plt
import scipy.ndimage.interpolation  as interplt
from PIL import Image
import lmdb
import pickle
import cv2

sys.path.append(os.path.dirname(os.path.abspath('')))
from utils.directory import DATA_DIR, DATASET_DIR
import utils.visualize as visual
import utils.convert_xyz_uvd as xyzuvd
import utils.prepare_data as pd

In [ ]:
save_prefix = 'train_fpha'
img_dataroot = os.path.join(DATA_DIR, save_prefix + '_256_img.lmdb')
img_env = lmdb.open(img_dataroot, readonly=True, lock=False, readahead=False, meminit=False)

uvd_gt_scaled_dataroot = os.path.join(DATA_DIR, save_prefix + '_uvd_gt_scaled.lmdb')
uvd_gt_scaled_env = lmdb.open(uvd_gt_scaled_dataroot, readonly=True, lock=False, readahead=False, meminit=False)

xyz_gt_dataroot = os.path.join(DATA_DIR, save_prefix + '_xyz_gt.lmdb')
xyz_gt_env = lmdb.open(xyz_gt_dataroot, readonly=True, lock=False, readahead=False, meminit=False)

xyz_gt_canon_dataroot = os.path.join(DATA_DIR, save_prefix + '_xyz_gt_canon.lmdb')
xyz_gt_canon_env = lmdb.open(xyz_gt_canon_dataroot, readonly=True, lock=False, readahead=False, meminit=False)

rot_mat_dataroot = os.path.join(DATA_DIR, save_prefix + '_rot_mat.lmdb')
rot_mat_env = lmdb.open(rot_mat_dataroot, readonly=True, lock=False, readahead=False, meminit=False)

keys_cache_file = os.path.join(DATA_DIR, save_prefix + '_keys_cache.p')
print('Read lmdb keys from cache: {}'.format(keys_cache_file))
keys = pickle.load(open(keys_cache_file, "rb"))



In [ ]:
train_pairs, test_pairs = pd.get_fpha_data_list_general('color', DATASET_DIR)

file_name = [i for i,j in train_pairs]
xyz_gt = [j for i,j in train_pairs]

In [ ]:
# read one image
idx = 0
key = file_name[idx]
img = pd.read_lmdb(key, img_env, np.uint32, (256, 256, 3))

fig, ax = plt.subplots()
ax.imshow(img)
ax.set_title('img')

In [ ]:
uvd_gt_scaled = pd.read_lmdb(key, uvd_gt_scaled_env, np.float32, (21, 3))

scoremap_gt = pd.create_multiple_gaussian_map(uvd_gt_scaled, (256, 256))

for i in range(21):
    fig, ax = plt.subplots()
    scoremap_show = cv2.cvtColor(scoremap_gt[:, :, i],cv2.COLOR_GRAY2RGB)
    scoremap_show = cv2.normalize(scoremap_show, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_8UC3)
    show = 0.5*scoremap_show + 0.5*img
    ax.imshow(show.astype('uint32'))
    ax.set_title('scoremap_gt')

In [ ]:
xyz_gt = pd.read_lmdb(key, xyz_gt_env, np.float32, (21, 3))
xyz_gt_canon = pd.read_lmdb(key, xyz_gt_canon_env, np.float32, (21, 3))
rot_mat = pd.read_lmdb(key, rot_mat_env, np.float32, (3, 3))
keypoint_scale = pd.get_keypoint_scale(xyz_gt)

coord_xyz_rec = np.matmul(xyz_gt_canon, rot_mat)*keypoint_scale
coord_xyz_rec_gt = xyz_gt - xyz_gt[0, :]
print(np.allclose(coord_xyz_rec, coord_xyz_rec_gt))

In [ ]:
scoremap_dataroot = os.path.join(DATA_DIR, save_prefix + '_scoremap_32.lmdb')
scoremap_env = lmdb.open(scoremap_dataroot, readonly=True, lock=False, readahead=False, meminit=False)
scoremap = pd.read_lmdb(key, scoremap_env, np.float32, (32, 32, 21))
scoremap_resized = pd.sk_resize(scoremap_gt, (32, 32)).astype('float32')
img_32 = pd.sk_resize(img, (32, 32))
for i in range(21):
    fig, ax = plt.subplots()
    scoremap_show = cv2.cvtColor(scoremap_resized[:, :, i],cv2.COLOR_GRAY2RGB)
    scoremap_show = cv2.normalize(scoremap_show, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_8UC3)
    show = 0.5*scoremap_show + 0.5*img_32
    ax.imshow(show.astype('uint32'))
    ax.set_title('scoremap_gt')